In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

**1. Differentiate Master report and duplicate report**

In [ ]:
pd.options.display.max_colwidth = 1000

In [ ]:
csv_file = "/content/drive/MyDrive/eclipse_platform.csv"

data = pd.read_csv(csv_file)
data

,Issue_id,Priority,Component,Duplicated_issue,Title,Description,Status,Resolution,Version,Created_time,Resolved_time
0,1,P3,Team,NaN,Usability issue with external editors (1GE6IRL),- Setup a project that contains a *.gif resource; \t- release project to CVS; \t- edit the *.gif resource with an external editor; e.g. PaintShop; \t- save and close external editor; \t- in Navigator open the icon resource and verify that your changes are there; \t- release project; \t\t-> nothing to release!; \t- in Navigator open the icon resource and verify that your changes are still there; ; \tProblem: because I never Refreshed from local; the workspace hasnt changed so Release didnt find anything.; \tHowever opening the resource with an external editor found the modified file on disk and showed the changes.; ; \tThe real problem occurs if Release actually finds something to release but you dont spot that some resources are missing.; \tThis is extremely error prone: one of my changes didnt made it into build 110 because of this!; ; NOTES:; EG (5/23/01 3:00:33 PM); \tRelease should do a refresh from local before doing the release.; \tMoving to VCM; ; ; KM (05/27/01 5:10:19 PM);...,CLOSED,FIXED,2,10/10/2001 21:34,2/9/2012 15:57
1,2,P5,Team,NaN,Opening repository resources doesnt honor type (1GE6I1Q),Opening repository resource always open the default text editor and doesnt; \thonor any mapping between resource types and editors.; \tAs a result it is not possible to view the contents of an image (*.gif file) in a sensible way.; ; NOTES:; EG (5/24/2001 5:13:45 AM); \tAgreed that this is needed eventually but not critical for June. \t; \tThere should also be a Java outliner when editing a java resource from the repository.; \tMoving to VCM.; ; KM (5/24/01 2:09:21 PM); \tIn general you cannot open a specialized editor on a remote resource because they can sometimes; \tbe dependent on other resources and expect these to be local. For example; an HTML page which; \treferences a graphic; the HTML editor is going to try to look for the graphic to display it.; ; \tYou cant be sure that the editor was written to accomodate remote or missing resources.; \tWe discussed this some time ago with Dave T. and felt it was safer to just use vanilla editors for now.; ; KM (5/24/01 4:42:09 PM); \...,RESOLVED,FIXED,2,10/10/2001 21:34,5/7/2002 10:33
2,3,P5,Team,NaN,Sync does not indicate deletion (1GIEN83),KM (10/2/2001 5:55:18 PM); \tThis PR about the deletion indicator in the sync viewer being too subtle.; \tThis would be even more true for someone with vision problems.; \tNE has good suggestion of using the label of the compare text viewer.; \tWe may also consider a better icon.; \tOn to the PR....; ; - delete file from project; - synchronize; - there is no indication that the file has been deleted from the workspace; - it looks the same as if the file is in the workspace but is empty; ; NOTES:; ; KM (8/15/2001 3:37:53 PM); \tIt should appear as a rightfacing - arrow in the sync view.; \tSent screen cap to NE requesting clarification.; ; NE (8/20/01 9:13:38 AM); \tThe - in the arrow is clearer when there are other arrows present too (the + and change ones).; \tOtherwise; its easy to miss the distinction; especially when the users attention is focussed; \ton the text compare panes.; \tId expect the Workspace file: label to indicate that it has been deleted.; \tThe Repository file: ...,RESOLVED,FIXED,2,10/10/2001 21:34,5/7/2010 10:28
3,4,P5,Team,NaN,need better error message if catching up over read-only resource (1GF69TF),- become synchronized with some project in a repository; - use a different Eclipse to make a change to a file resource within this project and release it to the repository; - in the original Eclipse mark this file resource as being read-only (select it; right-click -> Properties; change; OK); - select the file resource; right-click -> Team -> Synchronize with Stream; - in the subsequent comparison view select the file resource; right-click -> Catchu

In [ ]:
data = data.drop(columns=['Priority', 'Component', 'Status', 'Resolution', 'Version', 'Created_time', 'Resolved_time'])
data.head(1)

,Issue_id,Duplicated_issue,Title,Description
0,1,NaN,Usability issue with external editors (1GE6IRL),- Setup a project that contains a *.gif resource; \t- release project to CVS; \t- edit the *.gif resource with an external editor; e.g. PaintShop; \t- save and close external editor; \t- in Navigator open the icon resource and verify that your changes are there; \t- release project; \t\t-> nothing to release!; \t- in Navigator open the icon resource and verify that your changes are still there; ; \tProblem: because I never Refreshed from local; the workspace hasnt changed so Release didnt find anything.; \tHowever opening the resource with an external editor found the modified file on disk and showed the changes.; ; \tThe real problem occurs if Release actually finds something to release but you dont spot that some resources are missing.; \tThis is extremely error prone: one of my changes didnt made it into build 110 because of this!; ; NOTES:; EG (5/23/01 3:00:33 PM); \tRelease should do a refresh from local before doing the release.; \tMoving to VCM; ; ; KM (05/27/01 5:10:19 PM);...


In [ ]:
data = data.dropna(axis=0, subset=['Description'])
data.head(2)

,Issue_id,Duplicated_issue,Title,Description
0,1,NaN,Usability issue with external editors (1GE6IRL),- Setup a project that contains a *.gif resource; \t- release project to CVS; \t- edit the *.gif resource with an external editor; e.g. PaintShop; \t- save and close external editor; \t- in Navigator open the icon resource and verify that your changes are there; \t- release project; \t\t-> nothing to release!; \t- in Navigator open the icon resource and verify that your changes are still there; ; \tProblem: because I never Refreshed from local; the workspace hasnt changed so Release didnt find anything.; \tHowever opening the resource with an external editor found the modified file on disk and showed the changes.; ; \tThe real problem occurs if Release actually finds something to release but you dont spot that some resources are missing.; \tThis is extremely error prone: one of my changes didnt made it into build 110 because of this!; ; NOTES:; EG (5/23/01 3:00:33 PM); \tRelease should do a refresh from local before doing the release.; \tMoving to VCM; ; ; KM (05/27/01 5:10:19 PM);...
1,2,NaN,Opening repository resources doesnt honor type (1GE6I1Q),Opening repository resource always open the default text editor and doesnt; \thonor any mapping between resource types and editors.; \tAs a result it is not possible to view the contents of an image (*.gif file) in a sensible way.; ; NOTES:; EG (5/24/2001 5:13:45 AM); \tAgreed that this is needed eventually but not critical for June. \t; \tThere should also be a Java outliner when editing a java resource from the repository.; \tMoving to VCM.; ; KM (5/24/01 2:09:21 PM); \tIn general you cannot open a specialized editor on a remote resource because they can sometimes; \tbe dependent on other resources and expect these to be local. For example; an HTML page which; \treferences a graphic; the HTML editor is going to try to look for the graphic to display it.; ; \tYou cant be sure that the editor was written to accomodate remote or missing resources.; \tWe discussed this some time ago with Dave T. and felt it was safer to just use vanilla editors for now.; ; KM (5/24/01 4:42:09 PM); \...


In [ ]:
#Removing the fixed patterns by replacing them with " "
data["Description"]= data["Description"].str.replace("fixed in HEAD", "", case = False)
data["Description"]= data["Description"].str.replace("has been marked as readonly", " ", case = False)

In [ ]:
import re
import string

def clean_text_from_description(text):
    # "\[ \)" escaped character, ".*" macthes dot one or more,  "]" match this character
    text = re.sub('\[.*]\)', '', text) 

    # "\w*" matches alphanumeric & underscore, "\f" form feed character (0000 1100, 0C etc)
    text = re.sub('\w*\f\w*', '', text)

    # "\(\)  Matches a "(" character (char code 40)", "*?  Match 0 or more of the preceding token"
    text = re.sub('\(.*?\)', '', text)

    text = re.sub('\w*\d\w*', '', text)
    text = text.lower()

    
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    return text

step_1 = lambda x: clean_text_from_description(x)
data['Description'] = data['Description'].apply(clean_text_from_description)
data.loc[[8]]

,Issue_id,Duplicated_issue,Title,Description
8,9,NaN,VCM Implementation - disallow root resource to be passed (1G8436G),the implementation has to be changed because a root resource might be passed \twhere an iresource is expected see also notes km \tthis pr is very old it should either be fixed moved to future or obsolete


In [ ]:
def clean_text_from_description_and_title(text):
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\t', '', text)
    return text

step_2 = lambda x: clean_text_from_description_and_title(x)

data['Title'] = data['Title'].apply(clean_text_from_description_and_title)
data['Description'] = data['Description'].apply(clean_text_from_description_and_title)

data.loc[[8]]

,Issue_id,Duplicated_issue,Title,Description
8,9,NaN,VCM Implementation - disallow root resource to be passed (1G8436G),the implementation has to be changed because a root resource might be passed where an iresource is expected see also notes km this pr is very old it should either be fixed moved to future or obsolete


In [ ]:
import nltk

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
import gensim
from gensim.utils import simple_preprocess as SIMPLE_PREPROCESS  # Convert a document into a list of tokens
from nltk.stem import WordNetLemmatizer
from gensim.parsing.preprocessing import STOPWORDS as STOP_WORDS

In [ ]:
def lemmatize(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')

def removal_tokenization_lemmatization(text):
    result = []
    for token in SIMPLE_PREPROCESS(text):
        if token not in STOP_WORDS  and len(token) > 5:
            result.append(lemmatize(token))
    return result


data['Title'] = data['Title'].map(removal_tokenization_lemmatization)
data['Description'] = data['Description'].map(removal_tokenization_lemmatization)

In [ ]:
data.loc[[0]]

,Issue_id,Duplicated_issue,Title,Description
0,1,NaN,"[usability, external, editors]","[project, contain, resource, release, project, resource, external, editor, paintshop, external, editor, navigator, resource, verify, change, release, project, release, navigator, resource, verify, change, problem, refresh, workspace, change, release, open, resource, external, editor, modify, show, change, problem, occur, release, actually, release, resources, miss, extremely, change, release, refresh, release, move, comment, relate, set, optoinal, mandatory, default, set, consider, workspaces, network, drive, effect, refresh, complete, workspace, runnable, runnable, release, optimal, builders, responsible, maintain, invariants, derive, resources, releasable, builders, release, invariants, maintain, release, inconsistent, summary, ensure, consistency, release, itpvcmwinnt, internal, compare, document, fail]"


In [ ]:
data.head(2)

,Issue_id,Duplicated_issue,Title,Description
0,1,NaN,"[usability, external, editors]","[project, contain, resource, release, project, resource, external, editor, paintshop, external, editor, navigator, resource, verify, change, release, project, release, navigator, resource, verify, change, problem, refresh, workspace, change, release, open, resource, external, editor, modify, show, change, problem, occur, release, actually, release, resources, miss, extremely, change, release, refresh, release, move, comment, relate, set, optoinal, mandatory, default, set, consider, workspaces, network, drive, effect, refresh, complete, workspace, runnable, runnable, release, optimal, builders, responsible, maintain, invariants, derive, resources, releasable, builders, release, invariants, maintain, release, inconsistent, summary, ensure, consistency, release, itpvcmwinnt, internal, compare, document, fail]"
1,2,NaN,"[open, repository, resources, doesnt]","[open, repository, resource, default, editor, doesnt, map, resource, editors, result, possible, content, sensible, agree, need, eventually, critical, outliner, edit, resource, repository, move, general, specialize, editor, remote, resource, dependent, resources, expect, example, reference, graphic, editor, graphic, display, editor, write, accomodate, remote, miss, resources, discuss, vanilla, editors, itpvcmall, repositories, editor]"


In [ ]:
duplicated_reports = data.dropna(axis=0, subset=['Duplicated_issue'])
duplicated_reports.reset_index(drop=True)
# duplicated_reports.to_csv('duplicated_reports.csv')
duplicated_reports

,Issue_id,Duplicated_issue,Title,Description
24,25,28126.0,"[default, connection]","[helpful, notion, default, repository, connection, connect, project, wouldnt, everytime, create, create, default, minimum, have, default, repository, helpful]"
39,40,20.0,"[connect, stream, gcqpkw]","[connect, stream, directly, content, workspace, repository, wizard, currently, populate, workspace, stream, repository, repository, browser, expand, select, children, workspace, move, future, stream, container, project, operation, meaningful, repository, workspace, frequent, operation]"
47,48,22.0,"[future, project, reference]","[project, reference, flavour, project, project, version, project, stream, currently, ensure, future, require, change, vcmmeta, format, solve, example, have, distinct, project, reference, store, different, example, nameprojectname, versionname, move, reminder, project, reference]"
60,61,60.0,"[vcm_meta, show, change, gdqtgw]","[install, declipse, perspective, create, project, call, create, helloworld, example, method, print, console, project, create, connection, cvslnxtest, homecvs, connect, expand, stream, select, project, manage, release, catchuprelease, select, project, choose, release, connect, repository, repository, select, workspace, navigator, select, manage, release, observe, change, release, include, vcmmeta, severe, force, release, addition, typically, hide, navigator, package, outgo, change, change, kevinm, exist, resource, write, present, doesnt, cvsignore, get, classfiles, vcmmeta, compare]"
93,94,2.0,"[repositories, editor]","[browse, repositories, editor, associate, behave, navigator, associate, editor, associate, default, viewer, repeat, manage, connect, repository, location, browse, external, viewer, notice, editor, repositories, behave, open, navigator, launch, viewer, general, specialize, editor, remote, resource, dependent, resources, expect, example, reference, graphic, editor, graphic, display, editor, write, accomodate, remote, miss, resources, discuss, vanilla, editors, itpjuiall, open, repository, resources, doesnt]"
...,...,...,...,...
85119,423852,422971.0,"[workbench, mcontext]","[backport, initially, create, mcontext, check, mcontext, class, implement, example, mpartdescriptor, remove, mhandler, mpartdescriptor, stacktrace, follow]"
85122,423888,413977.0,"[keybindings, resize, content, assist, proposal, break, shortcuts]","[test, instal, standard, kepler, plugins, instal, trigger, completion, string, proposal, appear, resize, window, cancel, completion, press, escape, shortcuts, eclipse, cmdspace, application, eclipse]"
85132,424120,418254.0,"[window, context]","[create, attachment, context, eclipsebuildid, javaversion, corporation, bootloader, constants, nlruru, framework, arguments, product, commandline, arguments, product, problems, occur, invoke, plugin, orgeclipsejface]"
85137,424389,401709.0,"[viewmgmt, return, search, secondary]","[problem, method, secondary, parameters, call, iviewreference, viewref, return, console, internally, follow, log, iviewreference, viewrefs, loggertrace, loggertrace, console, log, secondary, mysecondaryid, append, secondary, match, indigo, environment, log, secondary, mysecondaryid, return, reference, thank]"


In [ ]:
master_reports = data[data.isnull().any(axis=1)]
master_reports.reset_index(drop=True)
master_reports.head(3)

,Issue_id,Duplicated_issue,Title,Description
0,1,NaN,"[usability, external, editors]","[project, contain, resource, release, project, resource, external, editor, paintshop, external, editor, navigator, resource, verify, change, release, project, release, navigator, resource, verify, change, problem, refresh, workspace, change, release, open, resource, external, editor, modify, show, change, problem, occur, release, actually, release, resources, miss, extremely, change, release, refresh, release, move, comment, relate, set, optoinal, mandatory, default, set, consider, workspaces, network, drive, effect, refresh, complete, workspace, runnable, runnable, release, optimal, builders, responsible, maintain, invariants, derive, resources, releasable, builders, release, invariants, maintain, release, inconsistent, summary, ensure, consistency, release, itpvcmwinnt, internal, compare, document, fail]"
1,2,NaN,"[open, repository, resources, doesnt]","[open, repository, resource, default, editor, doesnt, map, resource, editors, result, possible, content, sensible, agree, need, eventually, critical, outliner, edit, resource, repository, move, general, specialize, editor, remote, resource, dependent, resources, expect, example, reference, graphic, editor, graphic, display, editor, write, accomodate, remote, miss, resources, discuss, vanilla, editors, itpvcmall, repositories, editor]"
2,3,NaN,"[indicate, deletion]","[deletion, indicator, viewer, subtle, vision, problems, suggestion, compare, viewer, consider, better, delete, project, synchronize, indication, delete, workspace, workspace, appear, rightfacing, screen, request, clarification, clearer, arrows, present, distinction, especially, attention, focus, compare, expect, workspace, indicate, delete, repository, delete, repository]"


In [ ]:
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.porter import *
import numpy as np
from gensim import corpora,models
import time
import pickle
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
#Creating a dictionary using gensim library
#the dictionary object is used to create a bag of words (BoW) corpus which further used as the input to topic modelling and other models as well

dictionary = gensim.corpora.Dictionary(master_reports['Description'])
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 1000:
        break


0 actually
1 builders
2 change
3 comment
4 compare
5 complete
6 consider
7 consistency
8 contain
9 default
10 derive
11 document
12 drive
13 editor
14 effect
15 ensure
16 external
17 extremely
18 fail
19 inconsistent
20 internal
21 maintain
22 mandatory
23 miss
24 modify
25 move
26 navigator
27 network
28 occur
29 open
30 optimal
31 problem
32 project
33 refresh
34 relate
35 release
36 resource
37 resources
38 responsible
39 runnable
40 set
41 show
42 summary
43 verify
44 workspace
45 workspaces
46 accomodate
47 agree
48 content
49 critical
50 dependent
51 discuss
52 display
53 doesnt
54 edit
55 editors
56 eventually
57 example
58 expect
59 general
60 graphic
61 map
62 need
63 outliner
64 possible
65 reference
66 remote
67 repositories
68 repository
69 result
70 sensible
71 specialize
72 vanilla
73 write
74 appear
75 arrows
76 attention
77 better
78 clarification
79 clearer
80 delete
81 deletion
82 distinction
83 especially
84 focus
85 indicate
86 indication
87 indicator
88 present
89 

In [ ]:
#Creating BoW using the the dictionary
#it is the corpus that contains the word id and its frequency in each document

bag_of_words_corpus = [dictionary.doc2bow(doc) for doc in master_reports['Description']]
bag_of_words_corpus[0]

[(0, 1),
 (1, 2),
 (2, 5),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 3),
 (14, 1),
 (15, 1),
 (16, 3),
 (17, 1),
 (18, 1),
 (19, 1),
 (20, 1),
 (21, 2),
 (22, 1),
 (23, 1),
 (24, 1),
 (25, 1),
 (26, 2),
 (27, 1),
 (28, 1),
 (29, 1),
 (30, 1),
 (31, 2),
 (32, 3),
 (33, 3),
 (34, 1),
 (35, 12),
 (36, 5),
 (37, 2),
 (38, 1),
 (39, 2),
 (40, 2),
 (41, 1),
 (42, 1),
 (43, 2),
 (44, 2),
 (45, 1)]

In [ ]:
#Printing the BoW for single document
bag_of_words_doc = bag_of_words_corpus[0]
for i in range(len(bag_of_words_doc)):
    print("{}. (\"{}\") word seen {} time.".format(i+1, dictionary[bag_of_words_doc[i][0]], bag_of_words_doc[i][1]))
                                               

1. ("actually") word seen 1 time.
2. ("builders") word seen 2 time.
3. ("change") word seen 5 time.
4. ("comment") word seen 1 time.
5. ("compare") word seen 1 time.
6. ("complete") word seen 1 time.
7. ("consider") word seen 1 time.
8. ("consistency") word seen 1 time.
9. ("contain") word seen 1 time.
10. ("default") word seen 1 time.
11. ("derive") word seen 1 time.
12. ("document") word seen 1 time.
13. ("drive") word seen 1 time.
14. ("editor") word seen 3 time.
15. ("effect") word seen 1 time.
16. ("ensure") word seen 1 time.
17. ("external") word seen 3 time.
18. ("extremely") word seen 1 time.
19. ("fail") word seen 1 time.
20. ("inconsistent") word seen 1 time.
21. ("internal") word seen 1 time.
22. ("maintain") word seen 2 time.
23. ("mandatory") word seen 1 time.
24. ("miss") word seen 1 time.
25. ("modify") word seen 1 time.
26. ("move") word seen 1 time.
27. ("navigator") word seen 2 time.
28. ("network") word seen 1 time.
29. ("occur") word seen 1 time.
30. ("open") word s

In [ ]:
corpus = bag_of_words_corpus

#Training the LDA model on the BoW corpus
lda_model = gensim.models.LdaMulticore(corpus, num_topics=10, id2word=dictionary, passes=20, workers=2, iterations=100)

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.108*"thread" + 0.053*"libjvmdylib" + 0.049*"javathread" + 0.048*"daemon" + 0.041*"threadblocked" + 0.033*"libsystembdylib" + 0.032*"libclientdylib" + 0.032*"wait" + 0.030*"lock" + 0.023*"worker"
Topic: 1 
Words: 0.137*"message" + 0.055*"exception" + 0.032*"resolve" + 0.030*"occur" + 0.030*"miss" + 0.028*"orgeclipseosgi" + 0.026*"fail" + 0.026*"arguments" + 0.023*"subentry" + 0.022*"orgeclipseui"
Topic: 2 
Words: 0.090*"eclipse" + 0.022*"problem" + 0.021*"reproduce" + 0.017*"application" + 0.017*"create" + 0.016*"run" + 0.014*"version" + 0.013*"windows" + 0.013*"browser" + 0.012*"information"
Topic: 3 
Words: 0.040*"plugin" + 0.031*"action" + 0.030*"extension" + 0.028*"target" + 0.021*"command" + 0.021*"property" + 0.020*"filter" + 0.016*"define" + 0.015*"source" + 0.015*"context"
Topic: 4 
Words: 0.080*"public" + 0.071*"import" + 0.057*"display" + 0.026*"static" + 0.021*"return" + 0.019*"snippet" + 0.016*"widget" + 0.016*"string" + 0.015*"column" + 0.015*"private"
To